In [ ]:
#imports
!pip install transformers datasets torch scikit-learn pandas matplotlib faiss-cpu sentence-transformers
import os
import json
import faiss
import torch
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sentence_transformers import SentenceTransformer

np.random.seed(42)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets pandas matplotlib
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset from Hugging Face Hub
# Replace 'your_dataset_name' with the actual dataset identifier if different.
dataset = load_dataset("Astris/LA-Times")

# Inspect the dataset splits (e.g., train, test, etc.)
print(dataset)

# For this example, we'll work with the 'train' split.
# Convert the Hugging Face dataset split to a Pandas DataFrame.
df = pd.DataFrame(dataset['train'])

# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Compute the distribution of records across dates
date_distribution = df['date'].value_counts().sort_index()

# Print the distribution to the console
print(date_distribution)

# Plot the distribution as a bar chart
plt.figure(figsize=(12, 6))
date_distribution.plot(kind='bar')
plt.xlabel('Date')
plt.ylabel('Number of Records')
plt.title('Distribution of Records Across Dates in LA Times Dataset')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'url', 'dateTime', 'text'],
        num_rows: 3623587
    })
})


In [ ]:
df = pd.read_csv("Liar2_combined.csv", header = 0)

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')
df = df.dropna(subset=['date'])

print(df.head())


   label                                              title       date
0      1  90 percent of Americans "support universal bac... 2017-10-02
1      0  Last year was one of the deadliest years ever ... 2017-05-19
2      0  Bernie Sanders's plan is "to raise your taxes ... 2015-10-28
3      1  Voter ID is supported by an overwhelming major... 2021-12-08
4      0  Says Barack Obama "robbed Medicare (of) $716 b... 2012-08-12


In [ ]:
#Defining our Date Ranges
baseline_start, baseline_end = '2007-01-01', '2015-12-31'
update1_start, update1_end   = '2016-01-01', '2017-12-31'
update2_start, update2_end   = '2018-01-01', '2019-12-31'
update3_start, update3_end   = '2020-01-01', '2021-12-31'
update4_start, update4_end   = '2022-01-01', '2022-12-31'
test_start, test_end         = '2023-01-01', '2023-12-31'

# Baseline training set: entries w/ date <= split_date
baseline_df = df[(df['date'] >= baseline_start) & (df['date'] <= baseline_end)].copy()
update1_df = df[(df['date'] >= update1_start) & (df['date'] <= update1_end)].copy()
update2_df = df[(df['date'] >= update2_start) & (df['date'] <= update2_end)].copy()
update3_df = df[(df['date'] >= update3_start) & (df['date'] <= update3_end)].copy()
update4_df = df[(df['date'] >= update4_start) & (df['date'] <= update4_end)].copy()
test_df = df[(df['date'] >= test_start) & (df['date'] <= test_end)].copy()

# Display sample sizes for each block
print("Baseline samples:", len(baseline_df))
print("Update 1 samples:", len(update1_df))
print("Update 2 samples:", len(update2_df))
print("Update 3 samples:", len(update3_df))
print("Update 4 samples:", len(update4_df))
print("Test samples:", len(test_df))

Baseline samples: 10932
Update 1 samples: 3031
Update 2 samples: 2730
Update 3 samples: 3772
Update 4 samples: 1688
Test samples: 807


In [ ]:
print("Baseline distribution:")
print(baseline_df['label'].value_counts())

Baseline distribution:
label
1    6147
0    4785
Name: count, dtype: int64


#Faiss Index Creations


In [ ]:
real_articles_file = "News_Category_Dataset_v3.json"

real_articles = []
with open(real_articles_file, 'r') as f:
    for line in f:
        try:
            art = json.loads(line)
            art_date = None
            if 'date' in art:
                try:
                    art_date = datetime.strptime(art['date'], '%Y-%m-%d')
                except Exception as e:
                    print(f"Error parsing date for article: {art.get('date')}, {e}")
            art['parsed_date'] = art_date
            real_articles.append(art)
        except Exception as e:
            print("Error parsing line:", e)

# Helper function to filter articles up to a given end_date
def filter_articles_by_date(articles, end_date_str):
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    return [art for art in articles if art['parsed_date'] is not None and art['parsed_date'] <= end_date]

Error parsing line: Unterminated string starting at: line 1 column 177 (char 176)


In [ ]:
#Get Embeddings

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to build a FAISS index given a list of articles
def build_faiss_index(articles):
    headlines = [art['headline'].strip().lower() for art in articles]
    embeddings = embedding_model.encode(headlines, convert_to_numpy=True)
    faiss.normalize_L2(embeddings)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index, headlines

# Build cumulative FAISS indexes for each time period:
# Baseline (up to 2015-12-31)
articles_baseline = filter_articles_by_date(real_articles, baseline_end)
index_baseline, headlines_baseline = build_faiss_index(articles_baseline)
print("Baseline FAISS index built with", len(articles_baseline), "articles.")

# Update 1
articles_update1 = filter_articles_by_date(real_articles, update1_end)
index_update1, headlines_update1 = build_faiss_index(articles_update1)
print("Update 1 FAISS index built with", len(articles_update1), "articles.")

# Update 2
articles_update2 = filter_articles_by_date(real_articles, update2_end)
index_update2, headlines_update2 = build_faiss_index(articles_update2)
print("Update 2 FAISS index built with", len(articles_update2), "articles.")

# Update 3
articles_update3 = filter_articles_by_date(real_articles, update3_end)
index_update3, headlines_update3 = build_faiss_index(articles_update3)
print("Update 3 FAISS index built with", len(articles_update3), "articles.")

# Update 4
articles_update4 = filter_articles_by_date(real_articles, update4_end)
index_update4, headlines_update4 = build_faiss_index(articles_update4)
print("Update 4 FAISS index built with", len(articles_update4), "articles.")


Baseline FAISS index built with 130283 articles.
Update 1 FAISS index built with 192270 articles.
Update 2 FAISS index built with 204009 articles.
Update 3 FAISS index built with 208129 articles.
Update 4 FAISS index built with 209527 articles.


In [ ]:
sim_model = SentenceTransformer("all-MiniLM-L6-v2")
def get_top_k_similar_headlines(query_headline, k=3):
    query = query_headline.strip().lower()
    query_embedding = sim_model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)
    distances, indices = faiss_index.search(query_embedding, k)
    results = []
    for rank, idx in enumerate(indices[0]):
        if idx == -1:
            continue
        # Combine the retrieved headline with its distance value
        hybrid_fact = f"{headlines[idx]}"
        results.append(hybrid_fact)
    return results


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
test_query = "Over 4 million Americans get Omicron boosters"
# results = get_top_k_similar_headlines(test_query, embedding_model, index_update4, headlines_update4, k=3)

# print(results)


In [ ]:
# Save each FAISS index to disk
faiss.write_index(index_baseline, "faiss_index_baseline.index")
faiss.write_index(index_update1, "faiss_index_update1.index")
faiss.write_index(index_update2, "faiss_index_update2.index")
faiss.write_index(index_update3, "faiss_index_update3.index")
faiss.write_index(index_update4, "faiss_index_update4.index")

# Compress the index files into a single zip archive
!zip faiss_indexes.zip faiss_index_baseline.index faiss_index_update1.index faiss_index_update2.index faiss_index_update3.index faiss_index_update4.index

  adding: faiss_index_baseline.index (deflated 8%)
  adding: faiss_index_update1.index (deflated 7%)
  adding: faiss_index_update2.index (deflated 7%)
  adding: faiss_index_update3.index (deflated 7%)
  adding: faiss_index_update4.index (deflated 7%)


In [ ]:
import pickle

# To save the headlines:
headlines_data = {
    "baseline": headlines_baseline,
    "update1": headlines_update1,
    "update2": headlines_update2,
    "update3": headlines_update3,
    "update4": headlines_update4,
}

with open("faiss_headlines.pkl", "wb") as f:
    pickle.dump(headlines_data, f)

print("Headlines saved to faiss_headlines.pkl")


Headlines saved to faiss_headlines.pkl


#Load Faiss Indexes


In [ ]:
import zipfile
import os
from google.colab import files
import faiss

zip_filename = "faiss_indexes.zip"
extract_dir = "faiss_indexes"
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Extracted {zip_filename} to {extract_dir}")

# Load the FAISS indexes from the extracted folder.
index_baseline = faiss.read_index(os.path.join(extract_dir, "faiss_index_baseline.index"))
index_update1 = faiss.read_index(os.path.join(extract_dir, "faiss_index_update1.index"))
index_update2 = faiss.read_index(os.path.join(extract_dir, "faiss_index_update2.index"))
index_update3 = faiss.read_index(os.path.join(extract_dir, "faiss_index_update3.index"))
index_update4 = faiss.read_index(os.path.join(extract_dir, "faiss_index_update4.index"))

print("FAISS indexes loaded successfully!")


Extracted faiss_indexes.zip to faiss_indexes
FAISS indexes loaded successfully!


In [ ]:
import pickle

with open("faiss_headlines.pkl", "rb") as f:
    headlines_data = pickle.load(f)

headlines_baseline = headlines_data["baseline"]
headlines_update1  = headlines_data["update1"]
headlines_update2  = headlines_data["update2"]
headlines_update3  = headlines_data["update3"]
headlines_update4  = headlines_data["update4"]

print("Headlines loaded successfully!")


Headlines loaded successfully!


#RAG Models


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
def compute_entailment_score_for_sample(headline, similar_headlines, entailment_model, entailment_tokenizer):
    """
    Given a headline and its list of similar headlines, compute the aggregated entailment score.
    Uses the maximum probability for the ENTAILMENT class from a model like roberta-large-mnli.
    """
    pairs = [(headline, sim) for sim in similar_headlines]
    device = next(entailment_model.parameters()).device
    encoded = entailment_tokenizer.batch_encode_plus(
        pairs,
        return_tensors='pt',
        truncation=True,
        padding=True
    )
    encoded = {key: value.to(device) for key, value in encoded.items()}
    with torch.no_grad():
        outputs = entailment_model(**encoded)
    logits = outputs.logits  # shape: (k, 3)
    probs = F.softmax(logits, dim=-1)
    # For roberta-large-mnli, index 2 corresponds to ENTAILMENT.
    entailment_probs = probs[:, 2]
    aggregated_score = torch.max(entailment_probs).item()  # Using max; you can try averaging as well.
    return aggregated_score

In [ ]:
def preprocess_hybrid_data(df, k, entailment_model, entailment_tokenizer):
    """
    Precompute the entailment scores for each sample.
    Expects df to have columns 'title' and 'label'.
    Returns a list of dictionaries with keys: 'headline', 'entailment_score', 'label'
    """
    precomputed_samples = []
    i = 0
    for idx in range(len(df)):
        i += 1
        if i % 1000 == 0:
            print(i)
        sample = df.iloc[idx]
        headline = sample['title']
        label = sample['label']
        similar_headlines = get_top_k_similar_headlines(headline, k)
        score = compute_entailment_score_for_sample(headline, similar_headlines, entailment_model, entailment_tokenizer)
        precomputed_samples.append({
            "headlines": headline,
            "entailment_scores": score,
            "label": label
        })
    return precomputed_samples

In [ ]:
class HybridFakeNewsPrecomputedDataset(Dataset):
    def __init__(self, precomputed_samples):
        self._data = precomputed_samples

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        # If idx is a list, return a dictionary of lists.
        if isinstance(idx, list):
            batch = [self._data[i] for i in idx]
            # Convert list of dicts to dict of lists.
            aggregated = {key: [d[key] for d in batch] for key in batch[0].keys()}
            return aggregated
        else:
            return self._data[idx]

    @property
    def column_names(self):
        return list(self._data[0].keys()) if len(self._data) > 0 else []







In [ ]:
import torch.nn as nn
class HybridFakeNewsClassifierPrecomputed(nn.Module):
    def __init__(self,
                 bert_model_name='bert-base-uncased',
                 entailment_threshold=0.8):
        """
        If the precomputed entailment score is above entailment_threshold,
        the sample is overridden to be classified as real news (label 1).
        """
        super(HybridFakeNewsClassifierPrecomputed, self).__init__()
        # BERT classifier for raw input.
        self.bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        self.bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=2)
        self.entailment_threshold = entailment_threshold
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, headlines, entailment_scores, labels=None):
        """
        headlines: list of strings (batch of raw headlines)
        entailment_scores: tensor of shape (batch_size,) containing the precomputed scores.
        labels: tensor of true labels.
        """
        device = next(self.bert_model.parameters()).device
        # Tokenize and classify headlines with BERT.
        encoded = self.bert_tokenizer(headlines, return_tensors="pt", truncation=True, padding=True)
        encoded = {key: value.to(device) for key, value in encoded.items()}
        bert_outputs = self.bert_model(**encoded)
        bert_logits = bert_outputs.logits  # shape: (batch_size, 2)

        # Clone logits for override.
        final_logits = bert_logits.clone()
        # Create mask for samples with high entailment scores.
        entailment_scores = entailment_scores.to(device)
        high_mask = entailment_scores > self.entailment_threshold  # shape: (batch_size,)

        if high_mask.sum() > 0:
            # For these samples, override logits to strongly favor "real news" (label 1).
            override = torch.tensor([-10.0, 10.0], device=device)
            final_logits[high_mask] = override.unsqueeze(0).expand(high_mask.sum(), -1)

        if labels is not None:
            loss = self.loss_fn(final_logits, labels.to(device))
            return {'loss': loss, 'logits': final_logits}
        return {'logits': final_logits}


In [ ]:
def collate_fn(batch):
    """
    Expects each item in batch to be a dict with keys: 'headline', 'entailment_score', 'label'
    """
    headlines = [item["headlines"] for item in batch]
    # Convert entailment scores to a tensor.
    entailment_scores = torch.tensor([item["entailment_scores"] for item in batch], dtype=torch.float)
    labels = torch.tensor([item["label"] for item in batch])
    return {"headlines": headlines, "entailment_scores": entailment_scores, "labels": labels}

In [ ]:
headlines = headlines_baseline
faiss_index = index_baseline


In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:

k = 3  # number of similar headlines to retrieve

    # Load the entailment and encoder models and tokenizers.
entailment_model_name = 'roberta-large-mnli'
entailment_tokenizer = AutoTokenizer.from_pretrained(entailment_model_name)
entailment_model = AutoModelForSequenceClassification.from_pretrained(entailment_model_name)

entailment_model.eval()
for param in entailment_model.parameters():
    param.requires_grad = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
entailment_model.to(device)
train_samples = preprocess_hybrid_data(baseline_df, k, entailment_model, entailment_tokenizer)
train_dataset = HybridFakeNewsPrecomputedDataset(train_samples)




Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [ ]:

headlines = headlines_update4
faiss_index = index_update4

test_samples = preprocess_hybrid_data(test_df, k, entailment_model, entailment_tokenizer)
test_dataset = HybridFakeNewsPrecomputedDataset(test_samples)

In [ ]:
model = HybridFakeNewsClassifierPrecomputed(bert_model_name='bert-base-uncased', entailment_threshold=0.8)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


HybridFakeNewsClassifierPrecomputed(
  (bert_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_f

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to predicted labels.
    preds = np.argmax(logits, axis=-1)

    # Overall accuracy.
    accuracy = accuracy_score(labels, preds)

    # F1 scores per class.
    f1_per_class = f1_score(labels, preds, average=None)

    # Confusion matrix.
    conf_matrix = confusion_matrix(labels, preds)

    # Full classification report.
    report = classification_report(labels, preds, output_dict=True)

    # For example, you can print the detailed report.
    print("Classification Report:")
    print(classification_report(labels, preds))
    print("Confusion Matrix:")
    print(conf_matrix)

    return {
        "accuracy": accuracy,
        "f1_per_class": f1_per_class.tolist(),
        "confusion_matrix": conf_matrix.tolist(),
        "report": report
    }

In [ ]:
training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        learning_rate=1e-5,
        per_device_train_batch_size=10,
        evaluation_strategy="steps",
        logging_steps=500,
        save_steps=500,
        remove_unused_columns=False,
    )

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
    )

trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Accuracy,F1 Per Class,Confusion Matrix,Report
500,0.766700,0.785810,0.832714,"[0.898572501878287, 0.5229681978798587]","[[598, 107], [28, 74]]","{'0': {'precision': 0.9552715654952076, 'recall': 0.8482269503546099, 'f1-score': 0.898572501878287, 'support': 705.0}, '1': {'precision': 0.4088397790055249, 'recall': 0.7254901960784313, 'f1-score': 0.5229681978798587, 'support': 102.0}, 'accuracy': 0.8327137546468402, 'macro avg': {'precision': 0.6820556722503662, 'recall': 0.7868585732165206, 'f1-score': 0.7107703498790728, 'support': 807.0}, 'weighted avg': {'precision': 0.8862058378348016, 'recall': 0.8327137546468402, 'f1-score': 0.8510983519305302, 'support': 807.0}}"
1000,0.770400,0.786271,0.796778,"[0.8722741433021807, 0.503030303030303]","[[560, 145], [19, 83]]","{'0': {'precision': 0.9671848013816926, 'recall': 0.7943262411347518, 'f1-score': 0.8722741433021807, 'support': 705.0}, '1': {'precision': 0.36403508771929827, 'recall': 0.8137254901960784, 'f1-score': 0.503030303030303, 'support': 102.0}, 'accuracy': 0.7967781908302355, 'macro avg': {'precision': 0.6656099445504955, 'recall': 0.8040258656654151, 'f1-score': 0.6876522231662419, 'support': 807.0}, 'weighted avg': {'precision': 0.8909502650823565, 'recall': 0.7967781908302355, 'f1-score': 0.8256039181377056, 'support': 807.0}}"


Trainer is attempting to log a value of "[0.898572501878287, 0.5229681978798587]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[598, 107], [28, 74]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9552715654952076, 'recall': 0.8482269503546099, 'f1-score': 0.898572501878287, 'support': 705.0}, '1': {'precision': 0.4088397790055249, 'recall': 0.7254901960784313, 'f1-score': 0.5229681978798587, 'support': 102.0}, 'accuracy': 0.8327137546468402, 'macro avg': {'precision': 0.6820556722503662, 'recall': 0.7868585732165206, 'f1-score': 0.7107703498790728, 'support': 807.0}, 'weighted avg': {'precision': 0.8862058378348016, 'recall': 0.8327137546468402, 'f1-scor

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90       705
           1       0.41      0.73      0.52       102

    accuracy                           0.83       807
   macro avg       0.68      0.79      0.71       807
weighted avg       0.89      0.83      0.85       807

Confusion Matrix:
[[598 107]
 [ 28  74]]


Trainer is attempting to log a value of "[0.8722741433021807, 0.503030303030303]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[560, 145], [19, 83]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9671848013816926, 'recall': 0.7943262411347518, 'f1-score': 0.8722741433021807, 'support': 705.0}, '1': {'precision': 0.36403508771929827, 'recall': 0.8137254901960784, 'f1-score': 0.503030303030303, 'support': 102.0}, 'accuracy': 0.7967781908302355, 'macro avg': {'precision': 0.6656099445504955, 'recall': 0.8040258656654151, 'f1-score': 0.6876522231662419, 'support': 807.0}, 'weighted avg': {'precision': 0.8909502650823565, 'recall': 0.7967781908302355, 'f1-sco

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.79      0.87       705
           1       0.36      0.81      0.50       102

    accuracy                           0.80       807
   macro avg       0.67      0.80      0.69       807
weighted avg       0.89      0.80      0.83       807

Confusion Matrix:
[[560 145]
 [ 19  83]]


TrainOutput(global_step=1094, training_loss=0.7730104073310029, metrics={'train_runtime': 54.5587, 'train_samples_per_second': 200.371, 'train_steps_per_second': 20.052, 'total_flos': 0.0, 'train_loss': 0.7730104073310029, 'epoch': 1.0})

In [ ]:
import os
import torch

# Define the checkpoint directory
checkpoint_dir = "checkpoint_dir"

# Create the directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

# Save the model's state dict to a file in the checkpoint directory
save_path = os.path.join(checkpoint_dir, "custom_model.bin")
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")


Model saved to checkpoint_dir/custom_model.bin


In [ ]:
# Initialize a new model instance with the same parameters
model = HybridFakeNewsClassifierPrecomputed(
    bert_model_name='bert-base-uncased',
    entailment_threshold=0.8
)

# Load the saved state dictionary into the model
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, "custom_model.bin")))
print("Model loaded successfully.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully.


In [ ]:
headlines = headlines_update1
faiss_index = index_update1
train_samples = preprocess_hybrid_data(update1_df, k, entailment_model, entailment_tokenizer)
train_dataset = HybridFakeNewsPrecomputedDataset(train_samples)


1000
2000
3000


In [ ]:
training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        learning_rate=1e-5,
        per_device_train_batch_size=10,
        evaluation_strategy="steps",
        logging_steps=100,
        save_steps=100,
        remove_unused_columns=False,
    )

trainer = Trainer(
    model=model,  # model loaded from checkpoint
    args=training_args,
    train_dataset=train_dataset,  # new or combined dataset
    eval_dataset=test_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Accuracy,F1 Per Class,Confusion Matrix,Report
100,0.724200,0.661645,0.840149,"[0.9029345372460497, 0.5473684210526316]","[[600, 105], [24, 78]]","{'0': {'precision': 0.9615384615384616, 'recall': 0.851063829787234, 'f1-score': 0.9029345372460497, 'support': 705.0}, '1': {'precision': 0.4262295081967213, 'recall': 0.7647058823529411, 'f1-score': 0.5473684210526316, 'support': 102.0}, 'accuracy': 0.8401486988847584, 'macro avg': {'precision': 0.6938839848675914, 'recall': 0.8078848560700875, 'f1-score': 0.7251514791493406, 'support': 807.0}, 'weighted avg': {'precision': 0.89387859382984, 'recall': 0.8401486988847584, 'f1-score': 0.8579930950505991, 'support': 807.0}}"
200,0.835100,0.605741,0.848823,"[0.9085457271364318, 0.5642857142857143]","[[606, 99], [23, 79]]","{'0': {'precision': 0.9634340222575517, 'recall': 0.8595744680851064, 'f1-score': 0.9085457271364318, 'support': 705.0}, '1': {'precision': 0.4438202247191011, 'recall': 0.7745098039215687, 'f1-score': 0.5642857142857143, 'support': 102.0}, 'accuracy': 0.8488228004956629, 'macro avg': {'precision': 0.7036271234883265, 'recall': 0.8170421360033375, 'f1-score': 0.7364157207110731, 'support': 807.0}, 'weighted avg': {'precision': 0.8977579288883795, 'recall': 0.8488228004956629, 'f1-score': 0.8650333091553002, 'support': 807.0}}"
300,0.721300,0.608607,0.842627,"[0.9041509433962264, 0.5605536332179931]","[[599, 106], [21, 81]]","{'0': {'precision': 0.9661290322580646, 'recall': 0.849645390070922, 'f1-score': 0.9041509433962264, 'support': 705.0}, '1': {'precision': 0.43315508021390375, 'recall': 0.7941176470588235, 'f1-score': 0.5605536332179931, 'support': 102.0}, 'accuracy': 0.8426270136307311, 'macro avg': {'precision': 0.6996420562359842, 'recall': 0.8218815185648727, 'f1-score': 0.7323522883071097, 'support': 807.0}, 'weighted avg': {'precision': 0.8987642948249736, 'recall': 0.8426270136307311, 'f1-score': 0.8607222870911708, 'support': 807.0}}"


Trainer is attempting to log a value of "[0.9029345372460497, 0.5473684210526316]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[600, 105], [24, 78]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9615384615384616, 'recall': 0.851063829787234, 'f1-score': 0.9029345372460497, 'support': 705.0}, '1': {'precision': 0.4262295081967213, 'recall': 0.7647058823529411, 'f1-score': 0.5473684210526316, 'support': 102.0}, 'accuracy': 0.8401486988847584, 'macro avg': {'precision': 0.6938839848675914, 'recall': 0.8078848560700875, 'f1-score': 0.7251514791493406, 'support': 807.0}, 'weighted avg': {'precision': 0.89387859382984, 'recall': 0.8401486988847584, 'f1-score

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90       705
           1       0.43      0.76      0.55       102

    accuracy                           0.84       807
   macro avg       0.69      0.81      0.73       807
weighted avg       0.89      0.84      0.86       807

Confusion Matrix:
[[600 105]
 [ 24  78]]


Trainer is attempting to log a value of "[0.9085457271364318, 0.5642857142857143]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[606, 99], [23, 79]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9634340222575517, 'recall': 0.8595744680851064, 'f1-score': 0.9085457271364318, 'support': 705.0}, '1': {'precision': 0.4438202247191011, 'recall': 0.7745098039215687, 'f1-score': 0.5642857142857143, 'support': 102.0}, 'accuracy': 0.8488228004956629, 'macro avg': {'precision': 0.7036271234883265, 'recall': 0.8170421360033375, 'f1-score': 0.7364157207110731, 'support': 807.0}, 'weighted avg': {'precision': 0.8977579288883795, 'recall': 0.8488228004956629, 'f1-sco

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.86      0.91       705
           1       0.44      0.77      0.56       102

    accuracy                           0.85       807
   macro avg       0.70      0.82      0.74       807
weighted avg       0.90      0.85      0.87       807

Confusion Matrix:
[[606  99]
 [ 23  79]]


Trainer is attempting to log a value of "[0.9041509433962264, 0.5605536332179931]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[599, 106], [21, 81]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9661290322580646, 'recall': 0.849645390070922, 'f1-score': 0.9041509433962264, 'support': 705.0}, '1': {'precision': 0.43315508021390375, 'recall': 0.7941176470588235, 'f1-score': 0.5605536332179931, 'support': 102.0}, 'accuracy': 0.8426270136307311, 'macro avg': {'precision': 0.6996420562359842, 'recall': 0.8218815185648727, 'f1-score': 0.7323522883071097, 'support': 807.0}, 'weighted avg': {'precision': 0.8987642948249736, 'recall': 0.8426270136307311, 'f1-sc

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.85      0.90       705
           1       0.43      0.79      0.56       102

    accuracy                           0.84       807
   macro avg       0.70      0.82      0.73       807
weighted avg       0.90      0.84      0.86       807

Confusion Matrix:
[[599 106]
 [ 21  81]]


TrainOutput(global_step=304, training_loss=0.8227246936998869, metrics={'train_runtime': 25.2281, 'train_samples_per_second': 120.144, 'train_steps_per_second': 12.05, 'total_flos': 0.0, 'train_loss': 0.8227246936998869, 'epoch': 1.0})

In [ ]:
headlines = headlines_update2
faiss_index = index_update2
train_samples = preprocess_hybrid_data(update2_df, k, entailment_model, entailment_tokenizer)
train_dataset = HybridFakeNewsPrecomputedDataset(train_samples)


1000
2000


In [ ]:
training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        learning_rate=1e-5,
        per_device_train_batch_size=10,
        evaluation_strategy="steps",
        logging_steps=100,
        save_steps=100,
        remove_unused_columns=False,
    )

trainer = Trainer(
    model=model,  # model loaded from checkpoint
    args=training_args,
    train_dataset=train_dataset,  # new or combined dataset
    eval_dataset=test_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Accuracy,F1 Per Class,Confusion Matrix,Report
100,0.722100,0.525485,0.878563,"[0.9289855072463769, 0.5811965811965812]","[[641, 64], [34, 68]]","{'0': {'precision': 0.9496296296296296, 'recall': 0.9092198581560283, 'f1-score': 0.9289855072463769, 'support': 705.0}, '1': {'precision': 0.5151515151515151, 'recall': 0.6666666666666666, 'f1-score': 0.5811965811965812, 'support': 102.0}, 'accuracy': 0.8785625774473358, 'macro avg': {'precision': 0.7323905723905724, 'recall': 0.7879432624113475, 'f1-score': 0.7550910442214791, 'support': 807.0}, 'weighted avg': {'precision': 0.8947141802160388, 'recall': 0.8785625774473358, 'f1-score': 0.8850270556267, 'support': 807.0}}"
200,0.811800,0.493194,0.890954,"[0.9375, 0.5728155339805825]","[[660, 45], [43, 59]]","{'0': {'precision': 0.9388335704125178, 'recall': 0.9361702127659575, 'f1-score': 0.9375, 'support': 705.0}, '1': {'precision': 0.5673076923076923, 'recall': 0.5784313725490197, 'f1-score': 0.5728155339805825, 'support': 102.0}, 'accuracy': 0.8909541511771994, 'macro avg': {'precision': 0.753070631360105, 'recall': 0.7573007926574886, 'f1-score': 0.7551577669902912, 'support': 807.0}, 'weighted avg': {'precision': 0.8918749092394171, 'recall': 0.8909541511771994, 'f1-score': 0.8914060526220811, 'support': 807.0}}"


Trainer is attempting to log a value of "[0.9289855072463769, 0.5811965811965812]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[641, 64], [34, 68]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9496296296296296, 'recall': 0.9092198581560283, 'f1-score': 0.9289855072463769, 'support': 705.0}, '1': {'precision': 0.5151515151515151, 'recall': 0.6666666666666666, 'f1-score': 0.5811965811965812, 'support': 102.0}, 'accuracy': 0.8785625774473358, 'macro avg': {'precision': 0.7323905723905724, 'recall': 0.7879432624113475, 'f1-score': 0.7550910442214791, 'support': 807.0}, 'weighted avg': {'precision': 0.8947141802160388, 'recall': 0.8785625774473358, 'f1-sco

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       705
           1       0.52      0.67      0.58       102

    accuracy                           0.88       807
   macro avg       0.73      0.79      0.76       807
weighted avg       0.89      0.88      0.89       807

Confusion Matrix:
[[641  64]
 [ 34  68]]


Trainer is attempting to log a value of "[0.9375, 0.5728155339805825]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[660, 45], [43, 59]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9388335704125178, 'recall': 0.9361702127659575, 'f1-score': 0.9375, 'support': 705.0}, '1': {'precision': 0.5673076923076923, 'recall': 0.5784313725490197, 'f1-score': 0.5728155339805825, 'support': 102.0}, 'accuracy': 0.8909541511771994, 'macro avg': {'precision': 0.753070631360105, 'recall': 0.7573007926574886, 'f1-score': 0.7551577669902912, 'support': 807.0}, 'weighted avg': {'precision': 0.8918749092394171, 'recall': 0.8909541511771994, 'f1-score': 0.8914060526220811, 

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       705
           1       0.57      0.58      0.57       102

    accuracy                           0.89       807
   macro avg       0.75      0.76      0.76       807
weighted avg       0.89      0.89      0.89       807

Confusion Matrix:
[[660  45]
 [ 43  59]]


TrainOutput(global_step=273, training_loss=0.787338159023187, metrics={'train_runtime': 20.8179, 'train_samples_per_second': 131.137, 'train_steps_per_second': 13.114, 'total_flos': 0.0, 'train_loss': 0.787338159023187, 'epoch': 1.0})

In [ ]:
headlines = headlines_update3
faiss_index = index_update3
train_samples = preprocess_hybrid_data(update3_df, k, entailment_model, entailment_tokenizer)
train_dataset = HybridFakeNewsPrecomputedDataset(train_samples)

1000
2000
3000


In [ ]:
training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        learning_rate=1e-5,
        per_device_train_batch_size=10,
        evaluation_strategy="steps",
        logging_steps=100,
        save_steps=100,
        remove_unused_columns=False,
    )

trainer = Trainer(
    model=model,  # model loaded from checkpoint
    args=training_args,
    train_dataset=train_dataset,  # new or combined dataset
    eval_dataset=test_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Accuracy,F1 Per Class,Confusion Matrix,Report
100,0.597400,0.466192,0.897150,"[0.9429553264604811, 0.4779874213836478]","[[686, 19], [64, 38]]","{'0': {'precision': 0.9146666666666666, 'recall': 0.9730496453900709, 'f1-score': 0.9429553264604811, 'support': 705.0}, '1': {'precision': 0.6666666666666666, 'recall': 0.37254901960784315, 'f1-score': 0.4779874213836478, 'support': 102.0}, 'accuracy': 0.8971499380421314, 'macro avg': {'precision': 0.7906666666666666, 'recall': 0.672799332498957, 'f1-score': 0.7104713739220645, 'support': 807.0}, 'weighted avg': {'precision': 0.8833209417596034, 'recall': 0.8971499380421314, 'f1-score': 0.8841861488671268, 'support': 807.0}}"
200,0.595200,0.452099,0.893432,"[0.9399441340782123, 0.5274725274725275]","[[673, 32], [54, 48]]","{'0': {'precision': 0.9257221458046767, 'recall': 0.9546099290780142, 'f1-score': 0.9399441340782123, 'support': 705.0}, '1': {'precision': 0.6, 'recall': 0.47058823529411764, 'f1-score': 0.5274725274725275, 'support': 102.0}, 'accuracy': 0.8934324659231723, 'macro avg': {'precision': 0.7628610729023384, 'recall': 0.7125990821860659, 'f1-score': 0.7337083307753699, 'support': 807.0}, 'weighted avg': {'precision': 0.8845528039557585, 'recall': 0.8934324659231723, 'f1-score': 0.8878101763659696, 'support': 807.0}}"
300,0.506000,0.454046,0.890954,"[0.9370529327610873, 0.5925925925925926]","[[655, 50], [38, 64]]","{'0': {'precision': 0.9451659451659452, 'recall': 0.9290780141843972, 'f1-score': 0.9370529327610873, 'support': 705.0}, '1': {'precision': 0.5614035087719298, 'recall': 0.6274509803921569, 'f1-score': 0.5925925925925926, 'support': 102.0}, 'accuracy': 0.8909541511771994, 'macro avg': {'precision': 0.7532847269689376, 'recall': 0.7782644972882771, 'f1-score': 0.7648227626768399, 'support': 807.0}, 'weighted avg': {'precision': 0.8966606558075939, 'recall': 0.8909541511771994, 'f1-score': 0.8935151945985266, 'support': 807.0}}"


Trainer is attempting to log a value of "[0.9429553264604811, 0.4779874213836478]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[686, 19], [64, 38]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9146666666666666, 'recall': 0.9730496453900709, 'f1-score': 0.9429553264604811, 'support': 705.0}, '1': {'precision': 0.6666666666666666, 'recall': 0.37254901960784315, 'f1-score': 0.4779874213836478, 'support': 102.0}, 'accuracy': 0.8971499380421314, 'macro avg': {'precision': 0.7906666666666666, 'recall': 0.672799332498957, 'f1-score': 0.7104713739220645, 'support': 807.0}, 'weighted avg': {'precision': 0.8833209417596034, 'recall': 0.8971499380421314, 'f1-sco

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       705
           1       0.67      0.37      0.48       102

    accuracy                           0.90       807
   macro avg       0.79      0.67      0.71       807
weighted avg       0.88      0.90      0.88       807

Confusion Matrix:
[[686  19]
 [ 64  38]]


Trainer is attempting to log a value of "[0.9399441340782123, 0.5274725274725275]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[673, 32], [54, 48]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9257221458046767, 'recall': 0.9546099290780142, 'f1-score': 0.9399441340782123, 'support': 705.0}, '1': {'precision': 0.6, 'recall': 0.47058823529411764, 'f1-score': 0.5274725274725275, 'support': 102.0}, 'accuracy': 0.8934324659231723, 'macro avg': {'precision': 0.7628610729023384, 'recall': 0.7125990821860659, 'f1-score': 0.7337083307753699, 'support': 807.0}, 'weighted avg': {'precision': 0.8845528039557585, 'recall': 0.8934324659231723, 'f1-score': 0.8878101

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       705
           1       0.60      0.47      0.53       102

    accuracy                           0.89       807
   macro avg       0.76      0.71      0.73       807
weighted avg       0.88      0.89      0.89       807

Confusion Matrix:
[[673  32]
 [ 54  48]]


Trainer is attempting to log a value of "[0.9370529327610873, 0.5925925925925926]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[655, 50], [38, 64]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9451659451659452, 'recall': 0.9290780141843972, 'f1-score': 0.9370529327610873, 'support': 705.0}, '1': {'precision': 0.5614035087719298, 'recall': 0.6274509803921569, 'f1-score': 0.5925925925925926, 'support': 102.0}, 'accuracy': 0.8909541511771994, 'macro avg': {'precision': 0.7532847269689376, 'recall': 0.7782644972882771, 'f1-score': 0.7648227626768399, 'support': 807.0}, 'weighted avg': {'precision': 0.8966606558075939, 'recall': 0.8909541511771994, 'f1-sco

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94       705
           1       0.56      0.63      0.59       102

    accuracy                           0.89       807
   macro avg       0.75      0.78      0.76       807
weighted avg       0.90      0.89      0.89       807

Confusion Matrix:
[[655  50]
 [ 38  64]]


TrainOutput(global_step=378, training_loss=0.5883431611237703, metrics={'train_runtime': 31.9042, 'train_samples_per_second': 118.229, 'train_steps_per_second': 11.848, 'total_flos': 0.0, 'train_loss': 0.5883431611237703, 'epoch': 1.0})

In [ ]:
headlines = headlines_update4
faiss_index = index_update4
train_samples = preprocess_hybrid_data(update4_df, k, entailment_model, entailment_tokenizer)
train_dataset = HybridFakeNewsPrecomputedDataset(train_samples)

1000


In [ ]:
training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        learning_rate=1e-5,
        per_device_train_batch_size=10,
        evaluation_strategy="steps",
        logging_steps=100,
        save_steps=100,
        remove_unused_columns=False,
    )

trainer = Trainer(
    model=model,  # model loaded from checkpoint
    args=training_args,
    train_dataset=train_dataset,  # new or combined dataset
    eval_dataset=test_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Accuracy,F1 Per Class,Confusion Matrix,Report
100,0.526900,0.452373,0.900867,"[0.9445214979195562, 0.5348837209302325]","[[681, 24], [56, 46]]","{'0': {'precision': 0.9240162822252375, 'recall': 0.9659574468085106, 'f1-score': 0.9445214979195562, 'support': 705.0}, '1': {'precision': 0.6571428571428571, 'recall': 0.45098039215686275, 'f1-score': 0.5348837209302325, 'support': 102.0}, 'accuracy': 0.9008674101610905, 'macro avg': {'precision': 0.7905795696840473, 'recall': 0.7084689194826866, 'f1-score': 0.7397026094248944, 'support': 807.0}, 'weighted avg': {'precision': 0.8902850686460518, 'recall': 0.9008674101610905, 'f1-score': 0.8927457194153294, 'support': 807.0}}"


Trainer is attempting to log a value of "[0.9445214979195562, 0.5348837209302325]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[681, 24], [56, 46]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'0': {'precision': 0.9240162822252375, 'recall': 0.9659574468085106, 'f1-score': 0.9445214979195562, 'support': 705.0}, '1': {'precision': 0.6571428571428571, 'recall': 0.45098039215686275, 'f1-score': 0.5348837209302325, 'support': 102.0}, 'accuracy': 0.9008674101610905, 'macro avg': {'precision': 0.7905795696840473, 'recall': 0.7084689194826866, 'f1-score': 0.7397026094248944, 'support': 807.0}, 'weighted avg': {'precision': 0.8902850686460518, 'recall': 0.9008674101610905, 'f1-sc

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.97      0.94       705
           1       0.66      0.45      0.53       102

    accuracy                           0.90       807
   macro avg       0.79      0.71      0.74       807
weighted avg       0.89      0.90      0.89       807

Confusion Matrix:
[[681  24]
 [ 56  46]]


TrainOutput(global_step=169, training_loss=0.578090351714185, metrics={'train_runtime': 12.654, 'train_samples_per_second': 133.396, 'train_steps_per_second': 13.355, 'total_flos': 0.0, 'train_loss': 0.578090351714185, 'epoch': 1.0})